In [4]:
from datetime import datetime, timedelta, timezone
import logging
import sys
import pandas as pd
import ccxt
import asyncio

In [5]:
async def safe_gather(coros, n=1):
    return coros

async def async_wrap(func):
    # Wrapper to run the function in a thread (non-blocking)
    async def wrapper(*args, **kwargs):
        return await asyncio.to_thread(func, *args, **kwargs)
    return wrapper


def to_csv(data, filename, mode='w'):
    data.to_csv(filename, mode=mode)

async def vol_index_history(currency, exchange,
                            end=(datetime.now(tz=timezone.utc).replace(minute=0, second=0, microsecond=0)),
                            start=(
                                  datetime.now(tz=timezone.utc).replace(minute=0, second=0, microsecond=0)) - timedelta(
                                days=30),
                            timeframe='1h',
                            dirname='') -> pd.DataFrame:
    max_mark_data = int(1000)
    resolution = int(exchange.describe()['timeframes'][timeframe]) * 60

    e = int(end.timestamp())
    s = int(start.timestamp())
    f = int(max_mark_data * resolution)
    start_times = [s + k * f for k in range(1 + int((e - s) / f)) if s + k * f < e]

    logging.info(f'calling {sys._getframe(1).f_code.co_name} {len(start_times)} times')
    volIndex_coros = [
        exchange.publicGetGetVolatilityIndexData(params={'currency': currency, 'resolution': resolution,
                                                         'start_timestamp': start_time * 1000,
                                                         'end_timestamp': (start_time + f - int(resolution)) * 1000})
        for start_time in start_times]
    print(volIndex_coros)
    volIndex = await safe_gather(volIndex_coros, n=1)
    volIndex = [y for x in volIndex for y in x['result']['data']]

    column_names = ['t', 'vol_index_o', 'vol_index_h', 'vol_index_l', 'vol_index_c']
    if not volIndex:
        return pd.DataFrame()

    ###### spot
    data = pd.DataFrame(columns=column_names, data=volIndex, dtype=float).astype(dtype={'t': 'int64'}).set_index(
        't') / 100
    data.index = [datetime.utcfromtimestamp(x / 1000).replace(tzinfo=timezone.utc) for x in data.index]
    data = data[~data.index.duplicated()].sort_index()
    if dirname != '':
        to_csv(data, dirname + '/' + currency + '_volIndex.csv', mode='a')

    return data

In [7]:
exchange = ccxt.deribit()
await vol_index_history('ETH', exchange, dirname='deribit', start=datetime(2024, 1, 1, tzinfo=timezone.utc), end=datetime(2024, 9, 1, tzinfo=timezone.utc))

[{'jsonrpc': '2.0', 'result': {'continuation': None, 'data': [['1704067200000', '70.15', '70.75', '70.14', '70.15'], ['1704070800000', '70.15', '70.15', '69.18', '69.18'], ['1704074400000', '69.18', '69.51', '68.99', '69.27'], ['1704078000000', '69.28', '69.46', '69.2', '69.31'], ['1704081600000', '69.31', '69.4', '69.19', '69.4'], ['1704085200000', '69.4', '69.44', '68.95', '68.95'], ['1704088800000', '68.95', '69.08', '68.55', '68.58'], ['1704092400000', '68.58', '68.61', '68.4', '68.4'], ['1704096000000', '68.4', '68.46', '67.67', '67.72'], ['1704099600000', '67.72', '67.87', '67.1', '67.11'], ['1704103200000', '67.11', '67.12', '66.35', '66.35'], ['1704106800000', '66.35', '66.35', '66.07', '66.18'], ['1704110400000', '66.18', '66.42', '66.14', '66.42'], ['1704114000000', '66.42', '66.73', '66.42', '66.55'], ['1704117600000', '66.55', '66.74', '66.29', '66.74'], ['1704121200000', '66.74', '67.53', '66.74', '67.11'], ['1704124800000', '67.11', '67.39', '67.09', '67.29'], ['170412840

,vol_index_o,vol_index_h,vol_index_l,vol_index_c
2024-01-01 00:00:00+00:00,0.7015,0.7075,0.7014,0.7015
2024-01-01 01:00:00+00:00,0.7015,0.7015,0.6918,0.6918
2024-01-01 02:00:00+00:00,0.6918,0.6951,0.6899,0.6927
2024-01-01 03:00:00+00:00,0.6928,0.6946,0.6920,0.6931
2024-01-01 04:00:00+00:00,0.6931,0.6940,0.6919,0.6940
...,...,...,...,...
2024-09-06 19:00:00+00:00,0.7264,0.7288,0.7249,0.7282
2024-09-06 20:00:00+00:00,0.7282,0.7404,0.7269,0.7404
2024-09-06 21:00:00+00:00,0.7404,0.7426,0.7354,0.7385
2024-09-06 22:00:00+00:00,0.7385,0.7385,0.7328,0.7328
